<div align="center">

# YOLOv8 simple object detector using a procedural approach

</div>

The YOLOv8 is a novel an powerful object detection algorithm that combines high accuracy and real-time detection speeds. This Colab notebook demonstrates how to implement a simple object detector using a procedural approach, allowing us to detect objects in both static images and videos.




## Importing libraries, modules and files

### importing files from my github repo

In [ ]:
!git clone https://github.com/mohamedamine99/Object-tracking-and-counting-using-YOLOV8

Cloning into 'Object-tracking-and-counting-using-YOLOV8'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 90 (delta 0), reused 14 (delta 0), pack-reused 75
Unpacking objects: 100% (90/90), 244.01 MiB | 8.68 MiB/s, done.
Updating files: 100% (49/49), done.


### importing modules

In [ ]:
import numpy as np
import random
import os
import cv2
import time

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()
from ultralytics import YOLO


Ultralytics YOLOv8.0.58 🚀 Python-3.9.16 torch-1.13.1+cu116 CPU
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.1/107.7 GB disk)


### preparing file paths and directories

In [ ]:
img_results_path = '/content/image_object_detection_results'
vid_results_path = '/content/video_object_detection_results'
test_imgs_path = '/content/Object-tracking-and-counting-using-YOLOV8/test imgs'
test_vids_path = '/content/Object-tracking-and-counting-using-YOLOV8/test vids'

if not os.path.isdir(img_results_path):
    os.makedirs(img_results_path)

if not os.path.isdir(vid_results_path):
    os.makedirs(vid_results_path)

### importing YOLOv8 object detection variants

In [ ]:
yolo_names = ['yolov8n.pt', 'yolov8m.pt', 'yolov8x.pt']

# load yolo models and store them in a dict:
yolov8_models = {}
for yolo_name in yolo_names:
    yolov8_models[yolo_name[:-3]] = YOLO(yolo_name)


  0%|          | 0.00/6.23M [00:00<?, ?B/s]

  0%|          | 0.00/49.7M [00:00<?, ?B/s]

  0%|          | 0.00/131M [00:00<?, ?B/s]

## Implementing object detection

### testing object detection on an image

In [ ]:
for yolo_name , yolo_model in yolov8_models.items():
    results = yolo_model('https://ultralytics.com/images/bus.jpg', verbose = True)
    results = results [0]
    a = results.plot()
    cv2.imwrite(yolo_name + '.jpg', a)

  0%|          | 0.00/476k [00:00<?, ?B/s]

image 1/1 /content/bus.jpg: 640x480 4 persons, 1 bus, 1 stop sign, 310.4ms
Speed: 3.4ms preprocess, 310.4ms inference, 22.7ms postprocess per image at shape (1, 3, 640, 640)

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /content/bus.jpg: 640x480 4 persons, 1 bus, 1175.6ms
Speed: 1.0ms preprocess, 1175.6ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

Found https://ultralytics.com/images/bus.jpg locally at bus.jpg
image 1/1 /content/bus.jpg: 640x480 5 persons, 1 bicycle, 1 bus, 4443.8ms
Speed: 0.9ms preprocess, 4443.8ms inference, 2.1ms postprocess per image at shape (1, 3, 640, 640)


### Performing object detection on multiple static images in a directory using different YOLOv8 variants

In [ ]:
for yolo_name , yolo_model in yolov8_models.items():
    print(f"Model : {yolo_name}")
    for img_name in os.listdir(test_imgs_path):
        print(f"Target image : {img_name}")
        print(img_name.split('.')[0])
        print("-------------------------")
        img = cv2.imread(os.path.join(test_imgs_path, img_name))
        results = yolo_model(img, verbose = True)
        results = results [0]
        result_img = results.plot()

        save_dir = os.path.join(img_results_path, yolo_name)
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        save_name = yolo_name + ' -- ' + img_name.split('.')[0] + '.jpg'
        save_file = os.path.join(img_results_path, save_dir, save_name) 
        print(f"Saving results to : {save_file}")
        cv2.imwrite(save_file, result_img)

### Performing object detection on multiple video files from a directory using different YOLOv8 variants

In [ ]:
for yolo_name , yolo_model in yolov8_models.items():
    print(".......................................")
    print(f"Model : {yolo_name}")
    print(".......................................")

    for vid_name in os.listdir(test_vids_path):
        video_path = os.path.join(test_vids_path, vid_name)
      
       # Open input video file
        cap = cv2.VideoCapture(video_path)
        print("----------------------------")

        # Get video name 
        print(f"Detecting object in {vid_name} : ")

        # Get frame dimensions and print information about input video file
        width  = int(cap.get(3) )  # get `width` 
        height = int(cap.get(4) )  # get `height` 
        print(f"resolution : {width}x{height}")
        #print(video_path)

        save_dir = os.path.join(vid_results_path, yolo_name)
        if not os.path.isdir(save_dir):
            os.makedirs(save_dir)

        save_name = yolo_name + ' -- ' + vid_name.split('.')[0] + '.avi'
        save_file = os.path.join(vid_results_path, save_dir, save_name)
        print('SAVING TO :' + save_file)

        # define an output VideoWriter  object
        out = cv2.VideoWriter(save_file,
                            cv2.VideoWriter_fourcc(*"MJPG"),
                            30,(width,height))

        # Check if the video is opened correctly
        if not cap.isOpened():
            print("Error opening video stream or file")

        # Read the video frames
        while cap.isOpened():
            ret, frame = cap.read()

            # If the frame was not read successfully, break the loop
            if not ret:
                print("Error reading frame")
                break

            # Run object detection on the frame and calculate FPS
            beg = time.time()
            results = yolo_model(frame, verbose = False)
            fps = 1 / (time.time() - beg)
            results = results [0]
            frame = results.plot()
            # Display FPS on frame
            frame = cv2.putText(frame,f"FPS : {fps:,.2f}" , 
                                (5,15), cv2.FONT_HERSHEY_COMPLEX, 
                            0.5,  (0,0,255), 1, cv2.LINE_AA)
            

            # append frame to the video file
            out.write(frame)
            
            # the 'q' button is set as the
            # quitting button you may use any
            # desired button of your choice

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

        # After the loop release the cap 
        cap.release()
        out.release()

## Preparing results for download

In [ ]:
!zip -r img_results.zip /content/image_object_detection_results

In [ ]:
!zip -r vid_results.zip /content/video_object_detection_results